In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from sklearn.model_selection import train_test_split
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Definir las carpetas de origen y destino
source_dir = "/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/dataset"
train_dir = "/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/dataset/train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/dataset/test"

# Crear las carpetas de entrenamiento y prueba si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)



In [ ]:
# Para cada subcarpeta en la carpeta de origen
for class_name in ['chavez', 'maduro', 'nofigura']:
    # Crear las carpetas correspondientes en las carpetas de entrenamiento y prueba
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Obtener la lista de imágenes en la subcarpeta
    images = os.listdir(os.path.join(source_dir, class_name))

    # Dividir las imágenes en conjuntos de entrenamiento y prueba
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Mover las imágenes a las carpetas de entrenamiento y prueba correspondientes
    for image in train_images:
        shutil.move(os.path.join(source_dir, class_name, image), os.path.join(train_dir, class_name, image))
    for image in test_images:
        shutil.move(os.path.join(source_dir, class_name, image), os.path.join(test_dir, class_name, image))

In [ ]:
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 600 images belonging to 3 classes.


In [ ]:
valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=16,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 150 images belonging to 3 classes.


# Modelo 1

## Se implementó una red neuronal convolucional (CNN) sencilla que recuerda a una versión reducida de VGG, tambien referida como Tiny VGG.

In [ ]:
model_1 = tf.keras.models.Sequential([tf.keras.layers.Conv2D(filters=10,kernel_size=3, activation="relu",input_shape=(224, 224, 3)),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2,
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(3, activation="softmax")
])


In [ ]:
# Compile our CNN
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=6,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Epoch 1/6
19/19 [==============================] - 226s 12s/step - loss: 2.4740 - accuracy: 0.6350 - val_loss: 0.2860 - val_accuracy: 0.9533
Epoch 2/6
19/19 [==============================] - 37s 2s/step - loss: 0.1047 - accuracy: 0.9783 - val_loss: 0.0675 - val_accuracy: 0.9800
Epoch 3/6
19/19 [==============================] - 36s 2s/step - loss: 0.0206 - accuracy: 0.9983 - val_loss: 0.0369 - val_accuracy: 0.9800
Epoch 4/6
19/19 [==============================] - 37s 2s/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0417 - val_accuracy: 0.9800
Epoch 5/6
19/19 [==============================] - 40s 2s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 6/6
19/19 [==============================] - 40s 2s/step - loss: 5.5508e-04 - accuracy: 1.0000 - val_loss: 0.0108 - val_accuracy: 0.9933


In [ ]:
model_1.save("/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/save_model/modelo_1_final.keras")

In [ ]:
model_1.save_weights("/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/save_model/modelo_1_peso_final.h5")

In [ ]:
model_1.save("/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/save_model/modelo_1_final2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Modelo 2

# Se implementó la arquitectura EfficientNet B0 como base y agregando capas personalizadas para el problema específico de clasificación


In [ ]:
import pathlib
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['chavez' 'maduro' 'nofigura']


In [ ]:
!pip install keras-tuner

In [ ]:
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Definir la función de construcción del modelo
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32,
            max_value=512,
            step=32),
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)))
        model.add(tf.keras.layers.Dropout(hp.Float('dropout_' + str(i), 0.0, 0.5)))

    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(
        hp.Choice('learning_rate',
            values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    return model

# Crear el sintonizador
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=3,
    directory='/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/my_dir7',
    project_name='best')

# Ejecutar el sintonizador
tuner.search(train_data, epochs=30, validation_data=valid_data)
best_model = tuner.get_best_models(num_models=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
480               |480               |units_0
0.15773           |0.15773           |dropout_0
0.0001            |0.0001            |learning_rate

Epoch 1/30
19/19 [==============================] - 263s 12s/step - loss: 0.7253 - accuracy: 0.8833 - val_loss: 1.4053 - val_accuracy: 0.3333
Epoch 2/30
19/19 [==============================] - 224s 12s/step - loss: 0.2997 - accuracy: 0.9983 - val_loss: 1.3586 - val_accuracy: 0.3333
Epoch 3/30
19/19 [==============================] - 225s 12s/step - loss: 0.2584 - accuracy: 1.0000 - val_loss: 1.3638 - val_accuracy: 0.3333
Epoch 4/30
19/19 [==============================] - 190s 10s/step - loss: 0.2553 - accuracy: 1.0000 - val_loss: 1.3663 - val_accuracy: 0.3333
Epoch 5/30
19/19 [==============================] - 173s 9s/step - loss: 0.2524 - accuracy: 1.0000 - val_loss: 1.3620 - val_accuracy: 0.3333
Epoch 6/30
19/1

In [ ]:
best_model.save("/content/drive/MyDrive/Colab Notebooks/clasificacion imagenes Redes neuronales/save_model/best_model_3_1.keras")

## funciones de interes

In [ ]:
# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into (224,224,3).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
# Reconfig pred_and_plot function to work with multi-class images
def pred_and_plot(model, filename, class_names=class_names):
  """
  Imports an image located at filename, makes a prediction with model
  and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Add in logic for multi-class & get pred_class name
  if len(pred[0]) > 1:
    pred_class = class_names[tf.argmax(pred[0])]
  else:
    pred_class = class_names[int(tf.round(pred[0]))]

  print('Prediction Probabilities : ', pred[0])

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);
